In [ ]:
%load_ext autoreload
%autoreload 2

In [43]:
import lmdb
import json
from datetime import datetime, timezone
import numpy as np
import os
import pandas as pd
import plotly.express as px
from purple_py.db import (
    read_strfy_db,
    get_content_for_embeddings,
    query_db_for_record,
    create_weaviate_record,
    add_npub_cross_ref,
    process_events,
    create_weaviate_user_class,
    create_weaviate_event_class,
)
from sentence_transformers import SentenceTransformer
import weaviate
from dotenv import load_dotenv
load_dotenv()

client = weaviate.Client(
    url="http://localhost:8080",
)


In [54]:
print(os.getenv("STRFRY_DB_PATH"))

None


In [55]:
client.schema.delete_class('User')
client.schema.delete_class('Event')


In [56]:
# define Event class before User
create_weaviate_event_class(client)
create_weaviate_user_class(client)

In [57]:
start_time = datetime.now()
print("Start time:", start_time.strftime("%Y-%m-%d %H:%M:%S"))

process_events(client)

end_time = datetime.now()
duration = end_time - start_time
print("End time:", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duration:", duration)


Start time: 2023-11-26 06:59:21
End time: 2023-11-26 08:18:21
Duration: 1:18:59.276228


In [ ]:
#vectorize_content
process_output = {}
process_output["event_id_list"] = []
process_output["content_list"] = []

process_output = read_strfy_db(
    client,
    process_fn=get_content_for_embeddings,
    process_output=process_output,
)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
process_output["embedding_list"] = embedding_model.encode(process_output["content_list"])

In [ ]:
process_output["pubkey_dict"] = {}
process_output = query_db_for_record(
    client=client,
    process_fn=create_weaviate_record,
    process_input=process_output,
)
add_npub_cross_ref(client)

In [ ]:
# get event by event id
# get all events for pubkey

event_query = {
  "where": {
    "operator": "Equal",
    "path": ["event_id"],
    "valueString": event_id
  }
}

event_response = (
    client.query
    .get("Event", ["created_at", "pubkey", "kind"])
    .with_where(event_query["where"])
    .do()
)
event = event_response['data']['Get']['Event']

In [58]:
# get all events for pubkey
pubkey = "17538dc2a62769d09443f18c37cbe358fab5bbf981173542aa7c5ff171ed77c4"  # elsat

pubkey_query = {
  "where": {
    "operator": "Equal",
    "path": ["pubkey"],
    "valueString": pubkey
  }
}

user_response = (
    client.query
    .get("User", ["pubkey"])
    # .get("User", [
    #     "hasCreated { ... on Event { event_id created_at pubkey kind content } }"
    # ])
    # .with_where(pubkey_query["where"])
    .do()
)

event_response = (
    client.query
    .get("Event", ["event_id"])
    # .get("User", [
    #     "hasCreated { ... on Event { event_id created_at pubkey kind content } }"
    # ])
    # .with_where(pubkey_query["where"])
    .do()
)

created_response = (
    client.query
    .get("User", [
        "hasCreated { ... on Event { event_id created_at pubkey kind content } }"
    ])
    .do()
)

# Extracting the events associated with the user
pubkey_events = user_response['data']['Get']['User']
events_output = event_response['data']['Get']['Event']
created_output = created_response['data']['Get']['User']


In [59]:
len(pubkey_events), len(events_output),  len(created_output)

(100, 100, 100)

In [60]:
created_output

[{'hasCreated': None},
 {'hasCreated': None},
 {'hasCreated': None},
 {'hasCreated': [{'content': 'https://cdn.zbd.gg/ugc-uploads/967f2074-1700228430455.jpeg',
    'created_at': '2023-11-17T13:40:32Z',
    'event_id': 'd956cbc02af0b3eea6e20c7c3179c4d880effd74e403c857e750c0f21ed6df44',
    'kind': 1,
    'pubkey': '920a8b9779327d49891b328e467e92961276327cdc625a08726844c9a6b1c39f'},
   {'content': 'https://cdn.zbd.gg/ugc-uploads/7cc1df64-1700224012665.jpeg',
    'created_at': '2023-11-17T12:26:54Z',
    'event_id': 'afe8837441117281ce4958252b461e74729f1ecf7b6c31e556ddce30bfd6c5f1',
    'kind': 1,
    'pubkey': '920a8b9779327d49891b328e467e92961276327cdc625a08726844c9a6b1c39f'}]},
 {'hasCreated': None},
 {'hasCreated': [{'content': 'Terrible Idea #697:\n\n@here and @everyone for nostr',
    'created_at': '2023-11-16T13:48:53Z',
    'event_id': '02c5c666356b887ea165d1e44df8680ba9f0e7e070312d92f3e82942ff31b80e',
    'kind': 1,
    'pubkey': 'ed2023dcfdc71ab592c352f4190fe3c5bf7efc82fbdc668b7

In [ ]:
# 1st pass: get all content > min_length, get embeddings
start_time = datetime.now()
print("Start time:", start_time.strftime("%Y-%m-%d %H:%M:%S"))

env = lmdb.open(path=strfry_path, max_dbs=10)
payload_db = env.open_db(b"rasgueadb_defaultDb__EventPayload")
id_db = env.open_db(b"rasgueadb_defaultDb__Event__id")

# record_count = 0
# pubkey_dict = {}
# event_dict = {}

text_kinds = [1, 31922, 31923]
content_list = []
event_id_list = []
client.batch.configure(batch_size=batch_size)
with client.batch as batch:
    with env.begin(db=id_db) as txn:
        cursor = txn.cursor(db=id_db)
        for key, value in cursor:
            pl = txn.get(value, db=payload_db)
            if pl is None:
                print(key, value)
                continue
            event_hex = key.hex()[:64]  # event_id
            event_json = json.loads(pl[1:].decode("utf-8"))
            if "content" not in event_json or len(event_json['content']) < min_content_length:
                continue
            if event_json["kind"] in text_kinds:
                event_id = event_json["id"]
                content = event_json['content']
                if event_id not in event_id_list:
                    content_list.append(content)
                    event_id_list.append(event_id)
end_time = datetime.now()
print("End time:", end_time.strftime("%Y-%m-%d %H:%M:%S"))

# Calculate and print the duration
duration = end_time - start_time
print("Duration:", duration)


In [ ]:
start_time = datetime.now()
print("Start time:", start_time.strftime("%Y-%m-%d %H:%M:%S"))

end_time = datetime.now()
duration = end_time - start_time
print("End time:", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duration:", duration)


In [ ]:
# 1st pass: get all content > min_length, get embeddings
content_list = []
event_id_list = []
offset = 0
while True:
    events_response = client.query.get(
        "Event",
        ["event_id", "content"]
        # ["event_id", "content", "_additional { id }"]
    ).with_limit(page_limit).with_offset(offset).do()

    events = events_response["data"]["Get"]["Event"]
    if events is None:
        break
    for event in events:
        event['event_id']
        content = event['content']
        if len(content) > min_content_length:
            content_list.append(event['content'])
            event_id_list.append(event['event_id'])
    if len(events) < page_limit:
        break
    offset += page_limit

In [ ]:
users = client.query.get("User", ["name", "pubkey", "hasCreated"]).do()
for user in users["data"]["Get"]["User"]:
    print(f"User: {user['name']} has created events: {user['hasCreated']}")


In [ ]:
event["_additional"]["id"]

In [ ]:
user_response["data"]["Get"]["User"][:5]

In [ ]:
embeddings = embedding_model.encode(content_list)

In [ ]:
embeddings[0]

In [ ]:
len(embeddings)

In [ ]:
len(content_list), len(event_id_list)

In [ ]:
# query pubkeys with > 5 events
offset = 0
event_counts = {}
while True:
    events_response = client.query.get(
        "Event",
        # ["event_id", "content", "pubkey"]
        ["event_id", "content", "pubkey", "_additional { id }"]
    ).with_limit(page_limit).with_offset(offset).do()

    events = events_response["data"]["Get"]["Event"]
    if events is None:
        break
    for event in events:
        if len(event['content']) > min_content_length:
            pubkey = event['pubkey']
            event_counts[pubkey] = event_counts.get(pubkey, 0) + 1
    if len(events) < page_limit:
        break
    offset += page_limit

pubkeys_with_events = [
    pubkey for pubkey, count in event_counts.items() if count >= min_num_events
]

print(pubkeys_with_events)


In [ ]:
response = (
    client.query
    .get("Event", ["event_id", "pubkey", "kind", "content"])
    .with_limit(1)
    # .with_additional("vector")
    .with_additional(["distance", "id"])
    .do()
)
print(json.dumps(response, indent=4))

In [ ]:
response = (
    client.query
    .aggregate("Event")
    .with_meta_count()
    .do()
)

print(json.dumps(response, indent=2))

In [ ]:
near_text_config = {
  "concepts": ["hair typo"],
}

response = (
    client.query
    .get("Event", ["event_id", "content"])
    .with_near_text(near_text_config)
    .with_limit(100)
    .with_additional(["distance", "id"])
    .do()
)

result = response['data']['Get']['Event']
# print(json.dumps(response, indent=4))

In [ ]:
pubkey_value = "eab0e756d32b80bcd464f3d844b8040303075a13eabc3599a762c9ac7ab91f4f"

query = """
{
  Get {
    Event(
      where: {
        path: ["pubkey"]
        operator: Equal
        valueString: "%s"
      }
      limit: 10000
    ) {
      event_id
      created_at
      pubkey
      kind
      content
      _additional {
        vector
      }
    }
  }
}
""" % pubkey_value

result = client.query.raw(query)
events = result['data']['Get']['Event']

In [ ]:
vectors = [event['_additional']['vector'] for event in events]
vectors_array = np.array(vectors)
mean_vector = np.mean(vectors_array, axis=0)

# event_ids = [event['event_id'] for event in events]

# # Create a DataFrame
# df_vec = pd.DataFrame(vectors, index=event_ids, columns=[f'feature_{i}' for i in range(len(vectors[0]))])


In [ ]:
mean_vector[:5]

In [ ]:
response = (
    client.query
    .get("Event", ["question"])
    .do()
)

In [ ]:
def process_month(event_json):
    date = datetime.fromtimestamp(event_json['created_at'])
    month = date.strftime("%Y-%m")
    return month

def process_date(event_json):
    date = (
        datetime.fromtimestamp(event_json['created_at'])
        .strftime("%m-%d-%Y")
    )
    return date

def process_kind(event_json):
    return event_json['kind']

def process_db(process_fn):
    env = lmdb.open(path=os.getenv("STRFRY_DB_FOLDER"), max_dbs=10)
    payload_db = env.open_db(b"rasgueadb_defaultDb__EventPayload")
    id_db = env.open_db(b"rasgueadb_defaultDb__Event__id")
    output_list = []

    with env.begin(db=id_db) as txn:
        cursor = txn.cursor(db=id_db)
        for key, value in cursor:
            pl = txn.get(value, db=payload_db)
            if pl is None:
                print(key, value)
                continue
            # event_hex = key.hex()[:64]  # event_id
            event_json = json.loads(pl[1:].decode("utf-8"))
            output_list.append(process_fn(event_json))
    return output_list

In [ ]:
month_list = process_db(process_month)
event_counts = pd.Series(month_list).value_counts().sort_index()
fig = px.bar(event_counts, x=event_counts.index, y=event_counts.values, labels={'x': 'Month', 'y': 'Number of Events'})
fig.update_layout(title='Number of Events by Month', xaxis_title='Month', yaxis_title='Number of Events')
fig.show()

In [ ]:
date_list = process_db(process_date)
event_counts = pd.Series(date_list).value_counts().sort_index()
fig = px.bar(event_counts, x=event_counts.index, y=event_counts.values, labels={'x': 'Date', 'y': 'Number of Events'})
fig.update_layout(title='Number of Events by Date', xaxis_title='Date', yaxis_title='Number of Events')
fig.show()

In [ ]:
kind_list = process_db(process_kind)
counts = pd.Series(kind_list).value_counts().sort_index()
fig = px.bar(counts, x=list(map(str, counts.index)), y=counts.values, labels={'x': 'kind', 'y': 'Number of Events'})
fig.update_layout(title='Number of Events by kind', xaxis_title='kind', yaxis_title='Number of Events')
fig.show()
# 1, 30023


In [ ]:
start_time = datetime.now()
print("Start time:", start_time.strftime("%Y-%m-%d %H:%M:%S"))

end_time = datetime.now()
print("End time:", end_time.strftime("%Y-%m-%d %H:%M:%S"))

# Calculate and print the duration
duration = end_time - start_time
print("Duration:", duration)

In [ ]:
env = lmdb.open(path=os.getenv("STRFRY_DB_FOLDER"), max_dbs=10)
payload_db = env.open_db(b"rasgueadb_defaultDb__EventPayload")
id_db = env.open_db(b"rasgueadb_defaultDb__Event__id")


pubkey_counts = {}
with env.begin(db=id_db) as txn:
    with env.begin(db=payload_db) as tpl:
        for key, value in txn.cursor():
            pl = tpl.get(value)
            if pl is None:
                raise Exception("db corrupt!?")
            event_hex = key.hex()[:64]  # event_id
            event_json = json.loads(pl[1:].decode("utf-8"))  # event json
            pubkey = event_json['pubkey']
            if pubkey in pubkey_counts:
                pubkey_counts[pubkey] += 1
            else:
                pubkey_counts[pubkey] = 1

In [ ]:
top_pubkeys = sorted(pubkey_counts, key=pubkey_counts.get, reverse=True)[:5]

for pubkey in top_pubkeys:
    print(f"PubKey: {pubkey}, Count: {pubkey_counts[pubkey]}")